## Chloe's Explore Notebook for the NLP Group Project

### Imports

In [13]:
import pandas as pd
import numpy as np

# Scraping
import requests
from bs4 import BeautifulSoup

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
import nltk.sentiment

from wordcloud import WordCloud

# Regex
import re

# Time
from time import strftime

import unicodedata
import json
from pprint import pprint

# Visualizations
import matplotlib.pyplot as plt
import seaborn as sns

# Custom Imports
import acquire 
from prepare import *

# Turn off pink boxes for demo
import warnings
warnings.filterwarnings("ignore")

# pd.set_option('display.max_colwidth', -1)

### Acquire

In [2]:
# acquire.scrape_github_data()

In [3]:
# df = pd.DataFrame(acquire.scrape_github_data(),columns=['repo','language','readme_contents'])
# df.head()


In [4]:
df = pd.read_json('repo_readmes.json')

In [5]:
df.shape

(90, 3)

In [6]:
original = pd.read_json('repo_readmes.json')
print(original)

                             repo    language  \
0                 bitcoin/bitcoin         C++   
1         bitcoinbook/bitcoinbook    AsciiDoc   
2               bitcoinj/bitcoinj        Java   
3                    bitcoin/bips    Wikitext   
4         bitcoinjs/bitcoinjs-lib  TypeScript   
..                            ...         ...   
85              spesmilo/electrum      Python   
86  bitcoin-wallet/bitcoin-wallet        Java   
87        etotheipi/BitcoinArmory         C++   
88    bitcoin-dot-org/Bitcoin.org        HTML   
89               jgarzik/cpuminer           C   

                                      readme_contents  
0   Bitcoin Core integration/staging tree\n=======...  
1   Code Examples: ![travis_ci](https://travis-ci....  
2   [![Github Build Status](https://github.com/bit...  
3   People wishing to submit BIPs, first should pr...  
4   # BitcoinJS (bitcoinjs-lib)\n[![Github CI](htt...  
..                                                ...  
85  Electrum - Ligh

### Prepare

In [7]:
df.head()

,repo,language,readme_contents
0,bitcoin/bitcoin,C++,Bitcoin Core integration/staging tree\n=======...
1,bitcoinbook/bitcoinbook,AsciiDoc,Code Examples: ![travis_ci](https://travis-ci....
2,bitcoinj/bitcoinj,Java,[![Github Build Status](https://github.com/bit...
3,bitcoin/bips,Wikitext,"People wishing to submit BIPs, first should pr..."
4,bitcoinjs/bitcoinjs-lib,TypeScript,# BitcoinJS (bitcoinjs-lib)\n[![Github CI](htt...


In [8]:
# if you want to see the whole column contents:
# pd.set_option('display.max_colwidth', -1)

In [9]:
# how many language observations do we have?

df.language.value_counts(normalize = True)

Java          0.2
C++           0.2
Wikitext      0.1
TypeScript    0.1
C             0.1
AsciiDoc      0.1
Python        0.1
HTML          0.1
Name: language, dtype: float64

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90 entries, 0 to 89
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   repo             90 non-null     object
 1   language         90 non-null     object
 2   readme_contents  90 non-null     object
dtypes: object(3)
memory usage: 2.8+ KB


In [16]:
# combine all messages in single string by language

java_words = basic_clean(' '.join(df[df.language == 'Java'].readme_contents))
c_plus_plus__words = basic_clean(' '.join(df[df.language == 'C++'].readme_contents))
wiki_words = basic_clean(' '.join(df[df.language == 'Wikitext '].readme_contents))
type_words = basic_clean(' '.join(df[df.language == 'TypeScript'].readme_contents))
c_words = basic_clean(' '.join(df[df.language == 'C'].readme_contents))
ascii_words = basic_clean(' '.join(df[df.language == 'AsciiDoc'].readme_contents))
python_words = basic_clean(' '.join(df[df.language == 'Python'].readme_contents))
html_words = basic_clean(' '.join(df[df.language == 'HTML'].readme_contents))
all_words = basic_clean(' '.join(df.readme_contents))

In [18]:
# all_words

### Explore